In [1]:
# -*- coding: utf-8 -*-
"""과제5 CNN기반 영상분류문제

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Py2GvZc1GwqNbrR4fuQrHFy6zBEdF1xF
"""

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
import os
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
import random

(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train=x_train.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)

x_test=x_test.astype(np.float32)/255.0
y_test=tf.keras.utils.to_categorical(y_test,10)

x_val, _, y_val,_ = train_test_split(x_test, y_test, test_size=0.6, random_state=1)

input_shape = x_train.shape[1:]

170498071/170498071 [==============================] - 5s 0us/step


In [2]:
#동일조건 유지해야 하는 변수(두 모델 모두 동일하게 적용해야 함)
g_epoch = 70
g_batch = 64

#중요 : 아래함수 변경 불가!
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1)
   tf.random.set_seed(1)
   np.random.seed(1)
   random.seed(1)
   os.environ['TF_DETERMINISTIC_OPS'] = '1'

reset_random_seeds() #필수
   
print("reduced train/val size:", len(x_train), len(x_val), "input shape:", input_shape)

reduced train/val size: 50000 4000 input shape: (32, 32, 3)


In [3]:
# to make this notebook's output stable across runs

tf.__version__

from tensorflow.keras.layers import MaxPooling2D, Dropout, Conv2D

cnn=Sequential()
cnn.add(Conv2D(64,(3,3),activation='relu',input_shape=(32,32,3)))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(128,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Flatten())
cnn.add(Dense(1000,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10,activation='softmax'))

cnn.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
cnn.summary()

hist=cnn.fit(x_train, y_train, batch_size=g_batch, epochs=g_epoch,
             validation_data=(x_val,y_val), verbose=1)

g_org_res=cnn.evaluate(x_test,y_test,verbose=0)
print("Baseline 정확률은",g_org_res[1]*100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0

In [4]:
no_class = 10

# for transfer learning only
from tensorflow.keras.applications import VGG16

os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)

# for transfer learning only
transfermodel = VGG16(weights='imagenet',include_top=False,
                    input_shape=input_shape)
#base_model.trainable=False     # it's up to you

# your model architecture
model=Sequential()
# 전처리 레이어 추가/변경 가능
model.add(transfermodel)    # for transfer learning only
model.add(Flatten())        # for transfer learning only
model.add(Dense(1000,activation='relu')) # <<-- 변경가능
model.add(Dense(no_class, activation='softmax')) # <<-- activation은 변경가능

model.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy']) # <<-- 변경가능
model.summary()

hist=model.fit(x_train, y_train, batch_size=g_batch, epochs=g_epoch,
             validation_data=(x_val,y_val), verbose=1)

yours=model.evaluate(x_test,y_test,verbose=0)
print("Baseline vs yours: ",g_org_res[1]*100, yours[1]*100)

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 1000)              513000    
                                                                 
 dense_3 (Dense)             (None, 10)                10010     
                                                                 
Total params: 15,237,698
Trainable params: 15,237,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/70
782/782 [==============================] - 36s 40ms/step - loss: 0.9136 - accuracy: 0.6810 - val_l

In [5]:
org = g_org_res[1]*100
yours = yours[1]*100

if yours > (org + 2):
    print('SUCCESS! Difference: {0:0.3f}'.format(
                        (yours - org)))
else:
    print('TRY DIFFERENTLY! Difference: {0:0.3f}'.format(
                        (yours - org)))

SUCCESS! Difference: 10.030


In [7]:
import pandas as pd

# Save the Predict File
submission = pd.read_csv('Assignmnet_4_sample_solution.csv')

# 최종 제출 모델에 예측을 함.
y_predict = model.predict(x_test)

submission['label'] = np.argmax(y_predict,axis = 1)
submission.to_csv('submission.csv',index=False)

313/313 [==============================] - 3s 7ms/step
